### **Implementing Naive Bayes in pure Python using a MapReduce Approach**
Consider implementing a simple counting function where we sum the double of a range of numbers.

In [ ]:
numbers = list(range(1000))


In [ ]:
print (numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [ ]:
type(numbers)

list

In [ ]:
def doubled_sum(values):
    total = 0
    for n in numbers:
        total += n*2
    return total

We can print the outcome using




In [ ]:
print(doubled_sum(numbers))

999000


**MapReduce version of Example Problem**

As Python implements the functional programming paradigm too, it provides the functions required to implement the map-reduce paradigm builtin.

The foundation tools to implement map reduce are:

1.   "mapper" which is in charge of mapping each input value to a corresponding output value
2.   "reducer" which is in charge of merging multiple mapper outputs into a single output.

  
Both phases can be called multiple times (the output of a reducer can become the input of another reducer and a mapper can call other mappers).

Many MapReduce implementations also have additional phases like "combination" and "aggregation" which are executed after the mapper or the reducer to further modify their output.

In [ ]:
numbers = list(range(1000))

def mapper(value):
    return value*2

In [ ]:
print(mapper(numbers))

In [ ]:
def reducer(*values):
    return sum(values)

In [ ]:
#We can output the result of applying the map function as follows
first_step = map(mapper, numbers)
print (first_step)

#Next we can generate the output from the reducer and print it
result = reduce(reducer, map(mapper, numbers))
print (result)



The previous map-reduce in pure python implementation lacks of course the core feature of MapReduce: working parallely.

It's easy to understand that as each mapper and reducer works only on a subset of the data (its own input) it can work independently from the status of the other mappers and reducers. So the computation can proceed parallely.
Parallel Map Reduce in Pure Python

It's really easy to simulate a parallel map reduce in python by using the multiprocessing module

In [ ]:
import numpy as np  # Make numpy available using np.
from itertools import islice
import multiprocessing


class ParallelMapReduce(object):
    def __init__(self, map_func, reduce_func, num_workers=None):
        self.num_workers = num_workers
        self.map_func = map_func
        self.reduce_func = reduce_func
        self.pool = multiprocessing.Pool(num_workers)
    
    def partition(self, n, iterable):
        i = iter(iterable)
        piece = list(islice(i, n))
        while piece:
            yield piece
            piece = list(islice(i, n))
    
    def __call__(self, inputs):
        values = self.pool.map(self.map_func, inputs)
        
        print '>>> MAPPED VALUES (%s values): %s, ...' % (len(values), str(values[:10]))

        values = self.pool.map(self.reduce_func, 
                               self.partition(len(values)//self.num_workers, values))
        print '>>> REDUCED VALUES', values

        return self.reduce_func(values)

The previous mapreduce implementation takes a Mapper and a Reducer and splits them across num_workers until it gets back the final result

In [ ]:
numbers = range(1000)

def mapper(value):
    return value*2

def reducer(values):
    return sum(values)

mapreduce = ParallelMapReduce(mapper, reducer, 10)
print mapreduce(numbers)

Use the previous code fragments to do the following: 

1.   Implement a purely sequential version of Naive Bayes for Mapreduce
2.   Implement a ParallelMapReduce version of Naive Bayes.




In this project we will apply Naive Bayes to the well-known data set of mushroom data. This is found at

https://archive.ics.uci.edu/ml/datasets/mushroom

The zip-file of the data is given as part of the assignment.

Data Set Information:

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like ``leaflets three, let it be'' for Poisonous Oak and Ivy.

The dataset consists of 8124 training examples, each representing a single mushroom. The first column is the target variable containing the class labels, identifying whether the mushroom is poisonous or edible. The remaining columns are 22 discrete features that describe the mushroom in some observable way; their values are encoded by characters. For example, gill size is either broad (b) or narrow (n), and veil color can be brown (n), orange (o), white (w), or yellow (y). Each feature has numerous values, as described below.

Attribute Information:

1. cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
2. cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
3. cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
4. bruises?: bruises=t,no=f
5. odor: almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
6. gill-attachment: attached=a,descending=d,free=f,notched=n
7. gill-spacing: close=c,crowded=w,distant=d
8. gill-size: broad=b,narrow=n
9. gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y
10. stalk-shape: enlarging=e,tapering=t
11. stalk-root: bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
14. stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
15. stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
16. veil-type: partial=p,universal=u
17. veil-color: brown=n,orange=o,white=w,yellow=y
18. ring-number: none=n,one=o,two=t
19. ring-type: cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
20. spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
21. population: abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
22. habitat: grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d


The data description indicates that the feature stalk root has some missing values, denoted by ?. In this analysis, we'll read in the data and then exclude any training example that has missing values for stalk root.



In [ ]:
# Mushroom Data Set
# https://archive.ics.uci.edu/ml/datasets/Mushroom
# Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml].
# Irvine, CA: University of California, School of Information and Computer Science.

#We have provided a zip file with the data, but you can load it from the web if you want

import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_table('data/agaricus-lepiota.data', delimiter=',', header=None)
#exclude any training example that has missing values for stalk root
data = data[data['stalk root'] != '?']

You will need to train your classifier, and then evaluate the predictive accuracy on the test set.

In [ ]:

#first we must do a bit of data inspection and cleaning
y = data[:,0]
X = data[:,1:]


Cn=len(np.unique(y))

n,d = X.shape

print ("initial samples: {}".format(n))
print ("number of features: {}".format(d))
print ("number of class labels: {}".format(Cn))
print ()

print ("Class Labels are: {}".format(np.unique(y)))
print ()

print ("Take a look at unique outcomes per feature")
for i in range(0,d):
	print ("{}th: {}".format(i,np.unique(X[:,i])))

X = np.delete(X,(10,15),axis=1)

print ()
print ("Remove 10th feature because it has some missing data")
print ("Remove 15th feature because it is always 'p'")

#You do not need to use the following code, if you want

n,d = X.shape

# dictionary master list of unique features
featureDict = {}
for i in range(0,d):
	featureDict[i]= np.unique(X[:,i])

print ()
print ("After removing the two features")
print ("number of features: {}".format(d))

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
n_train = len(X_train)
n_test = len(X_test)

print ()
print ("number of training samples: {}".format(n_train))
print ("number of test samples: {}".format(n_test))

# Isolate the training set based on clasification label
X_train_e = X_train[y_train=='e']
X_train_p = X_train[y_train=='p']

# capture number of each class label in training set
n_train_e = len(X_train_e)
n_train_p = len(X_train_p)

# two dictionaries to capture likelihoods (features given class labels)
featureGivenEd = {}
featureGivenPo = {}





In [ ]:
# some helper code that may be useful for performance evaluation on the test set
print ()
print ("Probability of correct prediction")
print ((y_test_pred==y_test).sum()/n_test)